## 2주차 TF-IDF 간단 예시

In [ ]:
docs = [
  '먹고 싶은 사과', # 문서0 
  '먹고 싶은 바나나', # 문서1
  '길고 노란 바나나 바나나', # 문서2 
  '저는 과일이 좋아요' # 문서3 
]

### TF(Term Frequency)
- TF를 구성하는 Counter Vectorizer부터 보자!
- Counter Vectorizer는 `sklearn`의 패키지에서 가져와서 사용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() # Counter Vectorizer 객체 생성

- 문장을 Vector 형태로 바꾸는 Counter Vectorizer를 수행
- 이후, countvect를 출력하면 4x9 형태의 크기를 가지는 행렬 완성
- 여기서 4는 문서의 개수를 9는 단어의 개수

In [ ]:
# 문장을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(docs) 
countvect # 4x9 : 4개의 문서에 9개의 단어 

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

- toarray() 함수를 통해서 한번 문서들이 어떤 형태로 Vector화 되었는지 살펴보도록 하겠습니다.

In [ ]:
# toarray()를 통해서 문장이 Vector 형태의 값을 얻을 수 있음 
# 하지만, 각 인덱스와 컬럼이 무엇을 의미하는지에 대해서는 알 수가 없음 
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

In [ ]:
vect.vocabulary_

{'과일이': 0,
 '길고': 1,
 '노란': 2,
 '먹고': 3,
 '바나나': 4,
 '사과': 5,
 '싶은': 6,
 '저는': 7,
 '좋아요': 8}

In [ ]:
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [ ]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


###  TF-IDF (TF x IDF)

In [ ]:
# 동일한 방식으로 TF-IDF를 수행
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)

In [ ]:
tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


- 가장 많이 나온 단어 n 개만 사용하라는 `max_features` 파라미터들이 존재
- 희소문제 해결

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=4)
tfvect = vect.fit(docs)

tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,먹고,바나나,싶은
문서1,0.0,0.707107,0.00000,0.707107
문서2,0.0,0.577350,0.57735,0.577350
문서3,0.0,0.000000,1.00000,0.000000
문서4,1.0,0.000000,0.00000,0.000000


#  **영화 평점 데이터 기반 추천 시스템**
1. 영화 데이터셋 전처리 및 토크나이징
2. TF-IDF 생성 및 유사도 확인
---

**Dataset** 
* https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
* These files contain metadata for all 45,000 movies listed in the Full MovieLens Dataset. The dataset consists of movies released on or before July 2017.

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = ' '
data = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)

In [ ]:
data.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
data[['overview']]

,overview
0,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...
...,...
45461,Rising and falling between a man and woman.
45462,An artist struggles to finish his work while a...
45463,"When one of her hits goes wrong, a professiona..."
45464,"In a small town live two brothers, one a minis..."


### 전처리
- 다양한 정규표현식, 전처리 방법을 사용해보세요!

In [ ]:
# 전처리 
# overview의 결측치가 있는 항목은 모두 제거 
data = data[data['overview'].notnull()].reset_index(drop=True)

In [ ]:
data = data.loc[0:20000].reset_index(drop=True)

### TF-IDF

In [ ]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거 
tfidf = TfidfVectorizer(stop_words='english',max_features=10000)

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20001, 10000)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

- 참고 : enumerate는 for문을 돌 때 for문의 순서와 for문안의 값을 같이 출력해주는 함수입니다. 
- 예를 들어, enumerate(data['title'])의 경우는 (0, 'Toy Story'), (1, 'Jumanji')... 과 같이 출력이 됩니다.

In [ ]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
movie2id = {}
for i, c in enumerate(data['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [ ]:
movie2id

{0: 'Toy Story',
 1: 'Jumanji',
 2: 'Grumpier Old Men',
 3: 'Waiting to Exhale',
 4: 'Father of the Bride Part II',
 5: 'Heat',
 6: 'Sabrina',
 7: 'Tom and Huck',
 8: 'Sudden Death',
 9: 'GoldenEye',
 10: 'The American President',
 11: 'Dracula: Dead and Loving It',
 12: 'Balto',
 13: 'Nixon',
 14: 'Cutthroat Island',
 15: 'Casino',
 16: 'Sense and Sensibility',
 17: 'Four Rooms',
 18: 'Ace Ventura: When Nature Calls',
 19: 'Money Train',
 20: 'Get Shorty',
 21: 'Copycat',
 22: 'Assassins',
 23: 'Powder',
 24: 'Leaving Las Vegas',
 25: 'Othello',
 26: 'Now and Then',
 27: 'Persuasion',
 28: 'The City of Lost Children',
 29: 'Shanghai Triad',
 30: 'Dangerous Minds',
 31: 'Twelve Monkeys',
 32: 'Babe',
 33: 'Carrington',
 34: 'Dead Man Walking',
 35: 'Across the Sea of Time',
 36: 'It Takes Two',
 37: 'Clueless',
 38: 'Cry, the Beloved Country',
 39: 'Richard III',
 40: 'Dead Presidents',
 41: 'Restoration',
 42: 'Mortal Kombat',
 43: 'To Die For',
 44: 'How To Make An American Quilt',
 

In [ ]:
# Toy Story의 id 추출 
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 

In [ ]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Toy Story 3', 0.5365942551545341),
 ('Toy Story 2', 0.4517509299198836),
 ('The 40 Year Old Virgin', 0.3206177528182342),
 ('The Champ', 0.212986707946208),
 ('Rivers and Tides', 0.20763004515483832),
 ('Rebel Without a Cause', 0.19710416706857783),
 ('Class of 1984', 0.1896220036186201),
 ('For Your Consideration', 0.18576621846264357),
 ('Condorman', 0.18555568823708146),
 ('Man on the Moon', 0.16381563033083282)]